## Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from os import path
#%matplotlib notebook

from brainpipe.system import study
from brainpipe.feature import TF
from brainpipe.visual import *
from brainpipe.statistics import *

In [7]:
sf = 512
lines = [0]
f = (1,120,4,2)  # Frequency vector: (from, to, width, step) 1 to 120 Hz 4 2 #0.1, 13, 0.5, 0.1
baseline = [1408,1536] # Time in seconds (rest 700ms au milieu des 1s extraite) Odor (1408,1536) Rest 640,768
width, step = 50, 10 #Time vector
window_to_plot = [-250, 2000] # In seconds
exp = 'E'

### Compute TF on unbalanced data for all selected electrodes
#### No baseline correction - over 5.5s after odor is sent

In [8]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/Encoding_EpiPerf_LowHigh/')
path_respi = '/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/1bis_OE_BaseSam/JPlailly201306_seeg_ALS/behavior/respiration_amplitude/'
path2save = path.join (st.path, 'feature/0_Power_Encoding_EpiPerf_LowHigh/TF/all_trials/')
conds = ['low', 'high']
subjects = ['LEFC']#,'CHAF','VACJ','SEMC','FERJ','PIRJ','MICP'] #LEFC','CHAF','VACJ','SEMC','FERJ','MICP','PIRJ

for su in subjects:
    data0 = np.load(path.join(path_data, su+'_odor_'+conds[0]+'_bipo_sel_physFT.npz'))['x']
    data1 = np.load(path.join (path_data, su+'_odor_'+conds[1]+'_bipo_sel_physFT.npz'))['x']
    names = np.load(path.join(path_data, su+'_odor_'+conds[0]+'_bipo_sel_physFT.npz'))['Mai_RL']
    channels = np.load(path.join(path_data, su+'_odor_'+conds[0]+'_bipo_sel_physFT.npz'))['channels']
    print (su, 'low shape: ', data0.shape, 'high shape: ', data1.shape)
    all_data = np.concatenate((data0,data1),axis=2)
    print (su, 'all shape: ', all_data.shape)
    
    # ========================= COMPUTE TF FOR 1 ELEC =============================================
    for elec in [43]:#range(data0.shape[0]):
        data = all_data[elec,:,:]
        channel, label, sf, npts = channels[elec], names[elec], 512, data.shape[0]
        print (su, channel, label, data.shape, 'nb points', npts)

        time = 1000 * np.arange(-1024,data.shape[0]-1024) / sf #to set the 0 in your TF
        print ('Time points: ', len(time),min(time), max(time))
        tfObj = TF(sf, npts, f=f, time=time, width=width, step=step,baseline=baseline, norm=3)
        xtf_all, _ = tfObj.get(data, statmeth=None, maxstat=True, tail=2, metric='m_center')
        xtf_all = 100*np.squeeze(xtf_all)
        print ('TF size :', xtf_all.shape)
        
        timebin = np.array(tfObj.xvec)
        sl = slice(np.argmin(np.abs(timebin-window_to_plot[0])), np.argmin(np.abs(timebin-window_to_plot[1])))
        title = su+' Elec('+str(elec)+') '+channel+' '+label+' All trials'
        print('TF sel', xtf_all[:,sl].shape)
        tfObj.plot2D(plt.figure(), xtf_all[:, sl], xvec=tfObj.xvec[sl], yvec=tfObj.yvec, xlabel='Time (ms)',
                    ylabel='Frequency (hz)', cblabel='Power change (%)', cmap='viridis', 
                     title=title,pltargs={'shading':'gouraud'},vmin=-100, vmax=100, resample=(0.5, 0.1))
        ax = plt.gca()
        addLines(ax, vLines=lines, vColor=['firebrick','firebrick','black'], vWidth=[2]*3)
        
        #Add breathing to the plots
#         respi = np.mean(np.load(path_respi+su+'_sig_'+exp+'_prestim_resp.npy'),axis=0)[:200]
#         time_resp = np.load(path_respi+su+'_time_'+exp+'_prestim_resp.npy')[:200]
#         #Rescale breathing to fit TF plot
#         new_respi = (respi-respi.min()+0.1)*(100/(respi.max()-respi.min()))
#         ratio_time = 2000/200 #200 time points to plot in 2000ms
#         new_time = np.arange(-1500,500,ratio_time)
#         print('new time',ratio_time,new_time.shape,new_respi.shape)
#         ax.plot(new_time,new_respi,'black')       
        #Save all your plots
        fname = (path2save+su+'_Elec('+str(elec)+')_'+channel+'_'+label+'_All_trials.pdf')
        #print (fname)
        plt.savefig(fname, dpi=300, bbox_inches='tight')
        plt.clf()
        plt.close()
    del xtf_all, all_data, sf, channel
    


-> Olfacto loaded
LEFC low shape:  (60, 3584, 34) high shape:  (60, 3584, 21)
LEFC all shape:  (60, 3584, 55)
LEFC o4-o3 OFC (3584, 55) nb points 3584
Time points:  3584 -2000.0 4998.046875
TF size : (58, 354)
TF sel (58, 115)


### TF for all elecs - ONE CONDITION

In [6]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/Retrieval_No_Odor/')
path_respi = '/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/1bis_OE_BaseSam/JPlailly201306_seeg_ALS/behavior/respiration_amplitude/'
path2save = path.join (st.path, 'feature/TF_Inspi_No_Odor/Retrieval/')
conds = ['no_odor']
subjects = ['LEFC','CHAF','VACJ','SEMC','FERJ','PIRJ'] #LEFC','CHAF','VACJ','SEMC','FERJ','MICP','PIRJ

for su in subjects:
    data0 = np.load(path.join(path_data, su+'_odor_'+conds[0]+'_bipo_sel_phys.npz'))['x']
    names = np.load(path.join(path_data, su+'_odor_'+conds[0]+'_bipo_sel_phys.npz'))['Mai_RL']
    channels = np.load(path.join(path_data, su+'_odor_'+conds[0]+'_bipo_sel_phys.npz'))['channels']
    print (su, 'no odor shape: ', data0.shape)
    
    # ========================= COMPUTE TF FOR 1 ELEC =============================================
    for elec in range(data0.shape[0]):
        data = data0[elec,:,:]
        channel, label, sf, npts = channels[elec], names[elec], 512, data.shape[0]
        print (su, channel, label, data.shape, 'nb points', npts)

        time = 1000 * np.arange(-1536,data.shape[0]-1536) / sf #to set the 0 in your TF
        print ('Time points: ', len(time),min(time), max(time))
        tfObj = TF(sf, npts, f=f, time=time, width=width, step=step,baseline=baseline, norm=3)
        xtf_all, _ = tfObj.get(data, statmeth=None, maxstat=True, tail=2, metric='m_center')
        xtf_all = 100*np.squeeze(xtf_all)
        print ('TF size :', xtf_all.shape)
        
        timebin = np.array(tfObj.xvec)
        sl = slice(np.argmin(np.abs(timebin-window_to_plot[0])), np.argmin(np.abs(timebin-window_to_plot[1])))
        title = su+' Elec('+str(elec)+') '+channel+' '+label+' All trials'
        print('TF sel', xtf_all[:,sl].shape)
        tfObj.plot2D(plt.figure(), xtf_all[:, sl], xvec=tfObj.xvec[sl], yvec=tfObj.yvec, xlabel='Time (ms)',
                    ylabel='Frequency (hz)', cblabel='Power change (%)', cmap='viridis', 
                     title=title,pltargs={'shading':'gouraud'},vmin=-100, vmax=100, resample=(0.5, 0.1))
        ax = plt.gca()
        addLines(ax, vLines=lines, vColor=['firebrick','firebrick','black'], vWidth=[2]*3)
        
        #Add breathing to the plots
        respi = np.mean(np.load(path_respi+su+'_sig_'+exp+'_no_odor_resp.npy'),axis=0)[:200]
        time_resp = np.load(path_respi+su+'_time_'+exp+'_no_odor_resp.npy')[:200]
        #Rescale breathing to fit TF plot
        new_respi = (respi-respi.min()+0.2)*(100/(respi.max()-respi.min()))
        ratio_time = 2000/200 #200 time points to plot in 2000ms
        new_time = np.arange(0,2000,ratio_time)
        print('new time',ratio_time,new_time.shape,new_respi.shape)
        ax.plot(new_time,new_respi,'black')       
        #Save all your plots
        fname = (path2save+su+'_Elec('+str(elec)+')_'+channel+'_'+label+'_All_trials.png')
        #print (fname)
        plt.savefig(fname, dpi=300, bbox_inches='tight')
        plt.clf()
        plt.close()
    del xtf_all, data0, sf, channel, data
    


-> Olfacto loaded
LEFC no odor shape:  (148, 3584, 129)
LEFC a2-a1 Amg-PirT (3584, 129) nb points 3584
Time points:  3584 -3000.0 3998.046875
TF size : (58, 354)
TF sel (58, 116)
new time 10.0 (200,) (200,)
LEFC a6-a5 Ins (3584, 129) nb points 3584
Time points:  3584 -3000.0 3998.046875
TF size : (58, 354)
TF sel (58, 116)
new time 10.0 (200,) (200,)
LEFC a7-a6 Ins (3584, 129) nb points 3584
Time points:  3584 -3000.0 3998.046875
TF size : (58, 354)
TF sel (58, 116)
new time 10.0 (200,) (200,)
LEFC a8-a7 STG (3584, 129) nb points 3584
Time points:  3584 -3000.0 3998.046875
TF size : (58, 354)
TF sel (58, 116)
new time 10.0 (200,) (200,)
LEFC a9-a8 MTG (3584, 129) nb points 3584
Time points:  3584 -3000.0 3998.046875
TF size : (58, 354)
TF sel (58, 116)
new time 10.0 (200,) (200,)
LEFC a10-a9 MTG (3584, 129) nb points 3584
Time points:  3584 -3000.0 3998.046875
TF size : (58, 354)
TF sel (58, 116)
new time 10.0 (200,) (200,)
LEFC a11-a10 MTG (3584, 129) nb points 3584
Time points:  3584